In [ ]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta
import json
import gc

from data.cons_data import get_cons
from data.market_data import market_data

from utils.market_time import market_hours
from utils.params import PARAMS
from utils.clustering_methods import Clustering_methods

from pairs_finding.pairs_identification import cointegration_pairs
from pairs_finding.clustering import Clustering

from trade.pairs_trader import PairsTrader
from trade.optimizer import optimizer

import warnings

warnings.filterwarnings("ignore")

In [2]:
params = {
    ("GOOGL", "GOOG"): {
        PARAMS.beta_win: 100,
        PARAMS.beta_freq: "1d",  # TBC
        PARAMS.z_win_mean: 10,
        PARAMS.z_win_std: 10,
        PARAMS.z_entry: 2,
        PARAMS.z_exit: 1,
        PARAMS.trade_freq: "1m",
    },
    ("GOOGL", "AAPL"): {
        PARAMS.beta_win: 100,
        PARAMS.beta_freq: "1d",  # TBC
        PARAMS.z_win_mean: 10,
        PARAMS.z_win_std: 10,
        PARAMS.z_entry: 2,
        PARAMS.z_exit: 1,
        PARAMS.trade_freq: "5m",
    },
}

In [3]:
etf = "QQQ"
cons = get_cons(etf=etf)
cons_date = cons.read()

data = market_data(
    file_path="C:/Users/edmun/OneDrive/Desktop/Quantitative Trading Strategies/Project/qts/data/polygon/*.parquet"
)
out_path = "output/polygon"
earliest_date_year = [
    i
    for i in cons_date.keys()
    if datetime.strptime(i, "%Y-%m-%d").date()
    >= datetime.strptime("2020-06-30", "%Y-%m-%d").date()
]

period_ends = (
    pl.DataFrame(earliest_date_year, schema=["Date"])
    .with_columns(pl.all().cast(pl.Date))
    .with_columns(
        pl.all().dt.month().alias("Month"),
        pl.all().dt.year().alias("Year"),
    )
    .group_by(["Month", "Year"], maintain_order=True)
    .last()["Date"]
    .dt.strftime("%Y-%m-%d")
    .to_list()
)


In [ ]:
for i in range(2, len(period_ends)):  # range(2, len(period_ends))
    train_start, train_end, trade_end = (
        period_ends[i - 2],
        period_ends[i - 1],
        period_ends[i],
    )

    print(train_start, train_end, trade_end)
    # TRAINING PERIOD FINDING OPTIMAL PARAMS #
    data.read(cons=cons_date[train_end], start=train_start, end=train_end)

    train = data.filter(resample_freq="1h", hours=market_hours.MARKET)

    c = Clustering(df=train.select(pl.all().exclude(["date", "time"])))

    c.run_clustering(method=Clustering_methods.kmeans, min_clusters=2, max_clusters=6)

    find_pairs = cointegration_pairs(
        df=train.select(pl.all().exclude(["date", "time"])),
        p_val_cutoff=0.05,
        cluster_pairs=c.cluster_pairs,
    )
    find_pairs.identify_pairs()

    trader = PairsTrader(
        data=data,
        pairs=[],  # list(params.keys()),  # pairs_to_trade
        params=None,
        trade_hour=market_hours.MARKET,
    )

    opt = optimizer(
        backtester=trader,
        find_pairs=find_pairs,  # list(params.keys()), # pairs_to_trade
        start=pl.lit(train_start).str.strptime(pl.Date, "%Y-%m-%d"),
        end=pl.lit(train_end).str.strptime(pl.Date, "%Y-%m-%d"),
    )

    study = opt.optimize(n_trials=150)
    p = study.best_params

    study.trials_dataframe().to_csv(f"{out_path}/trials_{train_start}_{train_end}.csv")

    optimal_params = {}
    for key, value in p.items():
        if key != "pairs_to_trade":
            parts = key.split("_")

            pair = (parts[0], parts[1])
            param_name = "_".join(parts[2:])

            if pair not in optimal_params:
                optimal_params[pair] = {}

            optimal_params[pair][param_name] = value

    # TRADING PERIOD USING PARAMS
    # next trading day
    last_date = datetime.strptime(train_end, "%Y-%m-%d")
    next_day = (last_date + timedelta(days=1)).strftime("%Y-%m-%d")

    # reading pairs only from next trading day to next q end
    pairs_to_trade = list(optimal_params.keys())
    data.read(
        cons=set([item for pair in pairs_to_trade for item in pair]),
        start=train_start,
        end=trade_end,
    )

    trader.pairs = pairs_to_trade
    trader.params = optimal_params
    trader.update_data(data=data)

    pl_next_day = pl.lit(next_day).str.strptime(pl.Date, "%Y-%m-%d")
    pl_trade_end = pl.lit(trade_end).str.strptime(pl.Date, "%Y-%m-%d")
    returns = trader.backtest(
        start=pl_next_day, end=pl_trade_end, cost=0.0005, stop_loss=None
    )

    returns.with_columns(
        pl.col("CAPITAL").pct_change().fill_null(0).alias("PORT_RET")
    ).write_csv(f"{out_path}/result_{next_day}_{trade_end}.csv")

    convert_json = {f"{p1}_{p2}": params for (p1, p2), params in optimal_params.items()}
    with open(
        f"{out_path}/optimal_params_{next_day}_{trade_end}.json", "w"
    ) as json_file:
        json.dump(convert_json, json_file, default=str)

    del c, opt, find_pairs, trader  # free ram
    
    gc.collect()

2020-06-30 2020-07-31 2020-08-31


  File "c:\Users\edmun\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\edmun\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\edmun\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\edmun\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
[I 2025-05-31 18:22:40,330] A new study created in memory with name: no-name-fb775489-7495-41ad-bc84-8c4dece3a077


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2025-05-31 18:22:47,621] Trial 0 finished with value: 3.1532928118383805 and parameters: {'pairs_to_trade': 2, 'CPRT_SBUX_beta_win': 170, 'CPRT_SBUX_z_win_mean': 245, 'CPRT_SBUX_z_win_std': 410, 'CPRT_SBUX_z_entry': 1.9, 'CPRT_SBUX_z_exit': -3.4, 'CPRT_SBUX_trade_freq': '14m', 'FAST_SBUX_beta_win': 230, 'FAST_SBUX_z_win_mean': 265, 'FAST_SBUX_z_win_std': 160, 'FAST_SBUX_z_entry': 3.0, 'FAST_SBUX_z_exit': -2.6, 'FAST_SBUX_trade_freq': '7m', 'MRNA_TSLA_beta_win': 250, 'MRNA_TSLA_z_win_mean': 220, 'MRNA_TSLA_z_win_std': 425, 'MRNA_TSLA_z_entry': 1.4, 'MRNA_TSLA_z_exit': -1.0999999999999996, 'MRNA_TSLA_trade_freq': '8m'}. Best is trial 0 with value: 3.1532928118383805.
[I 2025-05-31 18:22:48,083] Trial 1 finished with value: -0.10575159021756912 and parameters: {'pairs_to_trade': 2, 'CPRT_SBUX_beta_win': 305, 'CPRT_SBUX_z_win_mean': 445, 'CPRT_SBUX_z_win_std': 450, 'CPRT_SBUX_z_entry': 1.9, 'CPRT_SBUX_z_exit': -2.0999999999999996, 'CPRT_SBUX_trade_freq': '11m', 'FAST_SBUX_beta_win': 460

[I 2025-05-31 18:24:05,830] A new study created in memory with name: no-name-6152ecaa-ffda-4248-a9b0-a2feb9175b90


...

[I 2025-05-31 18:36:54,979] Trial 0 finished with value: -5.4268794465745955 and parameters: {'pairs_to_trade': 2, 'AMGN_MSFT_beta_win': 170, 'AMGN_MSFT_z_win_mean': 245, 'AMGN_MSFT_z_win_std': 410, 'AMGN_MSFT_z_entry': 1.9, 'AMGN_MSFT_z_exit': -3.4, 'AMGN_MSFT_trade_freq': '14m', 'MU_TMUS_beta_win': 230, 'MU_TMUS_z_win_mean': 265, 'MU_TMUS_z_win_std': 160, 'MU_TMUS_z_entry': 3.0, 'MU_TMUS_z_exit': -2.6, 'MU_TMUS_trade_freq': '7m', 'SWKS_ZM_beta_win': 250, 'SWKS_ZM_z_win_mean': 220, 'SWKS_ZM_z_win_std': 425, 'SWKS_ZM_z_entry': 1.4, 'SWKS_ZM_z_exit': -1.0999999999999996, 'SWKS_ZM_trade_freq': '8m', 'CHTR_LRCX_beta_win': 180, 'CHTR_LRCX_z_win_mean': 305, 'CHTR_LRCX_z_win_std': 445, 'CHTR_LRCX_z_entry': 3.3000000000000003, 'CHTR_LRCX_z_exit': -2.6, 'CHTR_LRCX_trade_freq': '12m'}. Best is trial 0 with value: -5.4268794465745955.
[I 2025-05-31 18:36:55,554] Trial 1 finished with value: 1.4104438477386585 and parameters: {'pairs_to_trade': 5, 'AMGN_MSFT_beta_win': 460, 'AMGN_MSFT_z_win_mean'

[I 2025-05-31 18:38:20,104] A new study created in memory with name: no-name-4890df71-de01-4da6-abe4-048ee7990735


  0%|          | 0/150 [00:00<?, ?it/s]

[I 2025-05-31 18:38:20,339] Trial 0 finished with value: -2.1418783613383554 and parameters: {'pairs_to_trade': 2, 'CHTR_PEP_beta_win': 170, 'CHTR_PEP_z_win_mean': 245, 'CHTR_PEP_z_win_std': 410, 'CHTR_PEP_z_entry': 1.9, 'CHTR_PEP_z_exit': -3.4, 'CHTR_PEP_trade_freq': '14m', 'CPRT_KDP_beta_win': 230, 'CPRT_KDP_z_win_mean': 265, 'CPRT_KDP_z_win_std': 160, 'CPRT_KDP_z_entry': 3.0, 'CPRT_KDP_z_exit': -2.6, 'CPRT_KDP_trade_freq': '7m', 'AMAT_XLNX_beta_win': 250, 'AMAT_XLNX_z_win_mean': 220, 'AMAT_XLNX_z_win_std': 425, 'AMAT_XLNX_z_entry': 1.4, 'AMAT_XLNX_z_exit': -1.0999999999999996, 'AMAT_XLNX_trade_freq': '8m', 'AMAT_MTCH_beta_win': 180, 'AMAT_MTCH_z_win_mean': 305, 'AMAT_MTCH_z_win_std': 445, 'AMAT_MTCH_z_entry': 3.3000000000000003, 'AMAT_MTCH_z_exit': -2.6, 'AMAT_MTCH_trade_freq': '12m'}. Best is trial 0 with value: -2.1418783613383554.
[I 2025-05-31 18:38:20,940] Trial 1 finished with value: 3.1130992575651653 and parameters: {'pairs_to_trade': 5, 'CHTR_PEP_beta_win': 460, 'CHTR_PEP_z

IndexError: index 0 is out of bounds for axis 0 with size 0